In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_df = pd.read_excel(url)

In [3]:
shark_df = shark_df.drop(['pdf', 'href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22','Unnamed: 11','Source'], axis=1)

In [4]:
shark_df = shark_df.dropna(how='all')

In [15]:
shark_df = shark_df.dropna(subset=['Country', 'State', 'Location'])

In [16]:
shark_df.columns = shark_df.columns.str.lower()

In [17]:
shark_df.columns

Index(['date', 'year', 'type', 'country', 'state', 'location', 'activity',
       'name', 'sex', 'age', 'injury', 'time', 'species '],
      dtype='object')

In [22]:
shark_df.year.fillna(shark_df.year.mode()[0], inplace=True)

In [24]:
shark_df['year'].unique()

array([2024., 2023., 2022., 2021., 2020., 2019., 2018., 2017., 2016.,
       2015., 2014., 2013., 2012., 2011., 2010., 2009., 2008., 2007.,
       2006., 2005., 2004., 2003., 2002., 2001., 2000., 1999., 1998.,
       1997., 1996., 1995., 1984., 1994., 1993., 1992., 1991., 1990.,
       1989., 1988., 1987., 1986., 1985., 1983., 1982., 1981., 1980.,
       1979., 1978., 1977., 1976., 1975., 1974., 1973., 1972., 1971.,
       1970., 1969., 1968., 1967., 1966., 1965., 1964., 1963., 1962.,
       1961., 1960., 1959., 1958., 1957., 1956., 1955., 1954., 1953.,
       1952., 1951., 1950., 1949., 1948., 1848., 1947., 1946., 1945.,
       1944., 1943., 1942., 1941., 1940., 1939., 1938., 1937., 1936.,
       1935., 1934., 1933., 1932., 1931., 1930., 1929., 1928., 1927.,
       1926., 1925., 1924., 1923., 1922., 1921., 1920., 1919., 1918.,
       1917., 1916., 1915., 1914., 1913., 1912., 1911., 1910., 1909.,
       1908., 1907., 1906., 1905., 1904., 1903., 1902., 1901., 1900.,
       1899., 1898.,

In [25]:
result = shark_df['year'].unique()
longer_than_4 = [value for value in result if len(str(value)) > 4]
print(longer_than_4)

[2024.0, 2023.0, 2022.0, 2021.0, 2020.0, 2019.0, 2018.0, 2017.0, 2016.0, 2015.0, 2014.0, 2013.0, 2012.0, 2011.0, 2010.0, 2009.0, 2008.0, 2007.0, 2006.0, 2005.0, 2004.0, 2003.0, 2002.0, 2001.0, 2000.0, 1999.0, 1998.0, 1997.0, 1996.0, 1995.0, 1984.0, 1994.0, 1993.0, 1992.0, 1991.0, 1990.0, 1989.0, 1988.0, 1987.0, 1986.0, 1985.0, 1983.0, 1982.0, 1981.0, 1980.0, 1979.0, 1978.0, 1977.0, 1976.0, 1975.0, 1974.0, 1973.0, 1972.0, 1971.0, 1970.0, 1969.0, 1968.0, 1967.0, 1966.0, 1965.0, 1964.0, 1963.0, 1962.0, 1961.0, 1960.0, 1959.0, 1958.0, 1957.0, 1956.0, 1955.0, 1954.0, 1953.0, 1952.0, 1951.0, 1950.0, 1949.0, 1948.0, 1848.0, 1947.0, 1946.0, 1945.0, 1944.0, 1943.0, 1942.0, 1941.0, 1940.0, 1939.0, 1938.0, 1937.0, 1936.0, 1935.0, 1934.0, 1933.0, 1932.0, 1931.0, 1930.0, 1929.0, 1928.0, 1927.0, 1926.0, 1925.0, 1924.0, 1923.0, 1922.0, 1921.0, 1920.0, 1919.0, 1918.0, 1917.0, 1916.0, 1915.0, 1914.0, 1913.0, 1912.0, 1911.0, 1910.0, 1909.0, 1908.0, 1907.0, 1906.0, 1905.0, 1904.0, 1903.0, 1902.0, 1901.0,

In [14]:
shark_df['year'].isna().sum()

KeyError: ''